In [1]:
import os
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch_geometric
from torch_geometric.datasets import CoraFull, Planetoid, CitationFull
import torch_geometric.nn as gnn 
import torch_geometric.transforms as T

#from models import GPS
#from utils import test_model

torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/sujin/anaconda3/envs/latest0506/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

In [2]:
transform = T.Compose([T.NormalizeFeatures(), T.AddRandomWalkPE(16, 'RWPE')])
dataset = CitationFull(root='dataset/CoraML', name='Cora_ML', transform=transform)

In [ ]:
#transform = T.Compose([T.NormalizeFeatures(), T.AddRandomWalkPE(16, 'RWPE')])

: 

: 

In [ ]:
#dataset = CitationFull(root='dataset/Cora', name='Cora', transform=transform)

: 

In [2]:
#dataset = torch.load("/home/sujin/project/DS503/ds503/dataset/Cora/cora/RandomWalkPE.pt")

In [3]:
dataset

Cora_mlFull()

In [3]:
data = dataset[0]
df = pd.DataFrame(data.x.cpu())
df['y'] = data.y.cpu()
train, valid = train_test_split(df, stratify=df.y, test_size=0.33)
data.train_mask = torch.zeros(data.num_nodes, dtype=torch.bool)
data.train_mask[train.index]=True
data = data.to(device)

In [4]:
import random
import numpy as np

import torch
import torch.nn as nn

import torch_geometric.nn as gnn 

import torch
from torch.nn import Embedding, Linear, ModuleList, ReLU, Sequential
from torch_geometric.nn.conv import GINConv, GPSConv

class GPS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, pe_dim, number_of_classes, number_of_nodes, num_layers, heads=8, device = 'cuda:0'):
        super().__init__()

        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.pe_dim = pe_dim
        self.number_of_classes = number_of_classes
        self.number_of_nodes = number_of_nodes
        self.num_layers = num_layers
        self.heads=heads
        self.model = self.build_model().to(device)

    def forward(self, x, pe, edge_index):
        x = self.node_emb(x) + self.pe_lin(pe)
        return self.model(x, edge_index)
    
    def build_model(self):
        
        self.node_emb = Linear(self.in_channels, self.hidden_channels).to(device)
        self.pe_lin = Linear(self.pe_dim, self.hidden_channels).to(device)

        layers = []
        for _ in range(self.num_layers):
            net = Sequential(
                Linear(self.hidden_channels, self.hidden_channels),
                ReLU(),
                Linear(self.hidden_channels, self.hidden_channels),
            ).to(device)
            
            conv = GPSConv(channels = self.hidden_channels, conv = GINConv(net), heads=self.heads, attn_dropout=0.5)
            layers.append((conv,'x, edge_index -> x'))

        layers.append((Linear(self.hidden_channels, self.number_of_classes),'x -> x'))
        print(layers)

        return gnn.Sequential('x, edge_index', layers)
    
        
    def get_n_params(self):
        pp=0
        for p in list(self.model.parameters()):
            nn=1
            for s in list(p.size()):
                nn = nn*s
            pp += nn
        return pp

In [5]:
model = GPS(in_channels = data.x.shape[1],\
             hidden_channels=128,\
                pe_dim = 16,\
                  number_of_classes = len(data.y.unique()),\
                    number_of_nodes = data.x.shape[0],\
                        num_layers = 7,\
                          heads=4,\
                              device="cuda:0")
print(model.get_n_params())
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

[(GPSConv(128, conv=GINConv(nn=Sequential(
  (0): Linear(in_features=128, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
)), heads=4), 'x, edge_index -> x'), (GPSConv(128, conv=GINConv(nn=Sequential(
  (0): Linear(in_features=128, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
)), heads=4), 'x, edge_index -> x'), (GPSConv(128, conv=GINConv(nn=Sequential(
  (0): Linear(in_features=128, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
)), heads=4), 'x, edge_index -> x'), (GPSConv(128, conv=GINConv(nn=Sequential(
  (0): Linear(in_features=128, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=128, bias=True)
)), heads=4), 'x, edge_index -> x'), (GPSConv(128, conv=GINConv(nn=Sequential(
  (0): Linear(in_features=128, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, 

In [6]:
def train_model(model, data:torch_geometric.data.data.Data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.RWPE, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    pred = out.argmax(dim=-1)
    correct = pred[data.train_mask] == data.y[data.train_mask]
    acc = int(correct.sum()) / int(data.train_mask.sum())
    return loss

In [7]:
for epoch in range(500):
    loss = train_model(model, data, optimizer, criterion)
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

Epoch: 000, Loss: 1.9463
Epoch: 001, Loss: 1.7792
Epoch: 002, Loss: 1.6573
Epoch: 003, Loss: 1.5755
Epoch: 004, Loss: 1.5398
Epoch: 005, Loss: 1.5053
Epoch: 006, Loss: 1.4654
Epoch: 007, Loss: 1.4433
Epoch: 008, Loss: 1.4199
Epoch: 009, Loss: 1.4072
Epoch: 010, Loss: 1.4087
Epoch: 011, Loss: 1.3856
Epoch: 012, Loss: 1.3591
Epoch: 013, Loss: 1.3352
Epoch: 014, Loss: 1.3328
Epoch: 015, Loss: 1.3243
Epoch: 016, Loss: 1.3167
Epoch: 017, Loss: 1.2944
Epoch: 018, Loss: 1.2821
Epoch: 019, Loss: 1.2524
Epoch: 020, Loss: 1.2418
Epoch: 021, Loss: 1.2289
Epoch: 022, Loss: 1.1945
Epoch: 023, Loss: 1.1806
Epoch: 024, Loss: 1.1652
Epoch: 025, Loss: 1.1568
Epoch: 026, Loss: 1.1103
Epoch: 027, Loss: 1.1124
Epoch: 028, Loss: 1.0901
Epoch: 029, Loss: 1.0829
Epoch: 030, Loss: 1.0395
Epoch: 031, Loss: 1.0417
Epoch: 032, Loss: 1.0059
Epoch: 033, Loss: 1.0322
Epoch: 034, Loss: 1.0205
Epoch: 035, Loss: 1.0102
Epoch: 036, Loss: 0.9730
Epoch: 037, Loss: 0.9731
Epoch: 038, Loss: 0.9350
Epoch: 039, Loss: 0.9320


In [ ]:
  model.eval()
  out = model(data.x, data.RWPE, data.edge_index)
  pred = out.argmax(dim=-1)
  correct = (pred[~data.train_mask] == data.y[~data.train_mask])
  acc = int(correct.sum()) / int(correct.shape[0])
  f1 = f1_score(data.y[~data.train_mask.cpu()].cpu(), pred[~data.train_mask.cpu()].cpu(), average='micro')
  print(classification_report(data.y[~data.train_mask.cpu()].cpu(), pred[~data.train_mask.cpu()].cpu()))